# Getting Started

## Motivation

Data analysis workflows are often complex and involve many steps.
For example, we may want to:

1. Define or import functions to use for processing.
2. Define parameters for processing.
3. Load the data and apply functions to it.

There are a couple of problems with this:

- In complex workflows, we are forced to write a lot of boilerplate code to load data, apply functions, and save results.
  This is tedious and error-prone, e.g., since the order of function calls may be wrong or the wrong data and parameters may be passed to a function.
  This makes it hard to focus on the actual analysis.
- In Jupyter notebooks, the order of cell execution is not clear.
  This frequently leads to errors that are hard to track down in retrospect and analysis results that are hard to reproduce.

In traditional software development some of these problems are addressed by writing unit tests or integration tests.
However, in our experience this is challenging to do properly for data analysis workflows:

- Workflows are often interactive and under active development.
  Very frequently part of or all of the workflow is written in a Jupyter notebook.
- It is very time consuming to setup good test data for a workflow with good *fidelity*, i.e., test data that will actually allow you to catch errors in your workflow.

A very simplified model workflow of such as traditional is shown below:

In [ ]:
def load(filename: str) -> dict:
    """Load the data from the filename."""
    return {'data': [1, 2, float('nan'), 3], 'meta': {'filename': filename}}


def clean(raw_data: dict) -> list:
    """Clean the data, removing NaNs."""
    import math

    return [x for x in raw_data['data'] if not math.isnan(x)]


def process(data: list, param: float) -> float:
    """Process the data, multiplying the sum by the scale factor."""
    return sum(data) * param


filename = 'data.txt'
scale_factor = 2.0

raw_data = load(filename)
cleaned_data = clean(raw_data)
result = process(cleaned_data, scale_factor)
result

## Domain types, providers, and pipelines

Sciline uses a different approach.
We can rewrite the model workflow from above as follows:

In [ ]:
from typing import NewType
import sciline

# 1. Define domain types

Filename = NewType('Filename', str)
RawData = NewType('RawData', dict)
CleanedData = NewType('CleanedData', list)
ScaleFactor = NewType('ScaleFactor', float)
Result = NewType('Result', float)


# 2. Define providers


def load(filename: Filename) -> RawData:
    """Load the data from the filename."""
    return {'data': [1, 2, float('nan'), 3], 'meta': {'filename': filename}}


def clean(raw_data: RawData) -> CleanedData:
    """Clean the data, removing NaNs."""
    import math

    return [x for x in raw_data['data'] if not math.isnan(x)]


def process(data: CleanedData, param: ScaleFactor) -> Result:
    """Process the data, multiplying the sum by the scale factor."""
    return sum(data) * param


# 3. Create pipeline

providers = [load, clean, process]
params = {Filename: 'data.txt', ScaleFactor: 2.0}
pipeline = sciline.Pipeline(providers, params=params)

pipeline.compute(Result)

We can also visualize the task graph for computing `Result`:

In [ ]:
pipeline.visualize(Result, graph_attr={'rankdir': 'LR'})

Above, we have set up a data pipeline in three steps:

1. Define the domain types, i.e., unambiguous types specific to the workflow or problem.
   Sciline uses "domain types" as a way to identify inputs and outputs of processing steps.
2. Define the providers.
   Sciline uses "providers" to obtain domain objects, i.e., instances of required domain types.
   Each provider is either a callable (such as a function) that can compute the required domain object (from other domain objects) or simply a domain object.
3. Create the pipeline from a list of callables (that require domain objects and compute derived domain objects) and parameters (existing domain objects).

The pipeline sets up a directed acyclic graph (DAG) of processing steps.
This is done by inspecting the type-hints of the callables, i.e., the required domain objects.

If you are familiar with the concept of *dependency injection* you can think of the pipeline as a dependency injection container.
Each provider declares its dependencies (required domain objects) via *type hints* and the pipeline resolves them.
   
The pipeline can then be used to compute results or to visualize the structure of the workflow.
